In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import pathlib
import scipy.io as sio
from os.path import dirname, join as pjoin
import os
from shutil import copyfile
from sklearn.model_selection import train_test_split
import imageio
from PIL import Image

In [15]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

# Reduction de dimension des images

## whitening  equalization

# train test split 80% - 20%

In [17]:
creation = True
if not os.path.exists('train'):
    os.makedirs('train')
    creation = True
if not os.path.exists('test'):
    os.makedirs('test')
    creation = True

if (creation):
    for repertoir in os.listdir('Images'):
        if not os.path.exists('train/'+repertoir):
            os.makedirs('train/'+repertoir)
        if not os.path.exists('test/'+repertoir):
            os.makedirs('test/'+repertoir)
       
        tmp_data = os.listdir(pathlib.Path().joinpath('Images/'+repertoir).absolute())
        x_train, x_test = train_test_split(tmp_data, test_size = 0.2)
       
        for img1 in x_train:
            #img = reductionDimentsion(pathlib.Path().joinpath('Images/'+repertoir+'/'+img1).absolute())
            #Image.fromarray(img).convert('L').save(pathlib.Path().joinpath('train/'+repertoir+'/'+img1).absolute())
            copyfile(pathlib.Path().joinpath('Images/'+repertoir+'/'+img1).absolute(), pathlib.Path().joinpath('train/'+repertoir+'/'+img1).absolute())
        for img2 in x_test:
            #img = reductionDimentsion(pathlib.Path().joinpath('Images/'+repertoir+'/'+img2).absolute())
            #Image.fromarray(img).convert('L').save(pathlib.Path().joinpath('train/'+repertoir+'/'+img2).absolute())
            copyfile(pathlib.Path().joinpath('Images/'+repertoir+'/'+img2).absolute(), pathlib.Path().joinpath('test/'+repertoir+'/'+img2).absolute())

In [18]:
training_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [19]:
train = training_data.flow_from_directory(
    pathlib.Path().joinpath('Train').absolute(),
    target_size=(150,150),
    class_mode='categorical'
)

test = training_data.flow_from_directory(
    pathlib.Path().joinpath('test').absolute(),
    target_size=(150,150),
    class_mode='categorical'
)

Found 16418 images belonging to 120 classes.
Found 4162 images belonging to 120 classes.


(150, 150, 3)

# augmention du train 
### zoom, decalage 

# Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(120, activation='softmax')
    #3 neuronnes car 3 classes feuille papier ciseaux  
])

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

# Entrainement du model

In [ ]:
history = model.fit(train,
                    epochs=100, 
                   validation_data=train, 
                              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                               tfdocs.modeling.EpochDots()])

# Sauvegarde du model

In [ ]:
model.save('snooky.h5')

# Courbe d'apprentissage

In [ ]:
# summarize history for accuracy
plt.plot(history.history['mae'], label="train")
plt.plot(history.history['val_mae'], label="test")
plt.title('mean absolute error')
plt.ylim([0,10])
plt.ylabel('MAE [MPG]')
plt.xlabel('epoch')
plt.legend()
plt.show()
# summarize history for loss
plt.plot(history.history['mse'], label="train")
plt.plot(history.history['val_mse'], label="test")
plt.title('mean squarre error')
plt.ylabel('MSE [MPG^2]')
plt.xlabel('epoch')
plt.ylim([0,20])
plt.legend()
plt.show()